# Ex22



A diretora de um pessoal, elis, da companhia Aérea Boa Viagem deve decidir quantas novas aeromoças contratar e teinar nos próximos seis meses. As necessidades, expressas pelo núemero de aeromoças contratar e treinar nos próximos seis meses. As necessidades, expressas pelo número de aeromolas-horas-de-vôo necessários, são 8000 em janeiro; 9000 fevereiro; 7000 em março;10000 em abril; 9000 em maio e 11000 em junho.

Leva  um mês de treinamento antes que uma aeromoça possa ser posta num vôo regular, assim. uma garota deve ser contratada pelo menos um mês antes que ela seja realmente necessária. Cada moça treinada requer 100h de supervisão de uma aeromoça durante o mês de treinamento de modo que são diponíveis 100h a menos para serviçoes de vôo por aeromoças regurlares.

Cada aeromoça experiente pode trabalhar até 150h num mês, e boa viagem tem 60 aeromoças regulares disponíveis no começo de janeiro. Se o tempo máximo disponível das aeromoças experientes exceder as necessidades de vôo e treinamento de um mês, as garotas regulares trabalham menos que 150h e ninguém é dispensado. No fim de cada mês, aproximadamente10% das aeromoças experientes deixam seus empregos para se casarem ou por outras razões.

Uma aero moça experiente custa à companhia $\$1500$ e uma em treinamento $\$900$ por mês em salário e outros cargos

### **Formulação matemática**

### **Parametros**
  $M$ : Meses (Jan, fev,Mar,Abr,Mai,Jun)

  $H_m$: Horas de voo no mes $m \in M$

  $HT$: Horas de treinamento

  $HM$: Número máximo de horas de trabalho

  $Cx$ : Custo de uma aeromoça experiente

  $Cy$ : Custo aeromoça nova

### **Variáveis**
$x_{m}$ -Número de aeromoças experientes no mes $m \in M$

$y_{m}$ -Número de aeromoças novas no mes $m \in M$

### **Função objetivo**

$$
\text{Min} \quad  \sum_m CI x_{m} + CJ y_{m}
$$

### **Restrições**
1. Horas de vôo a ser atendida em cada mês, conisiderando os treinamentos
$$\sum_m 150 x_{m} - 100 y_{m}= H_m$$

2. Saida de aeromoças experientes, todos os meses temos uma redução de app 10% no quadro de aeromoças regulares. Foram utilizadas restrições de desigualdade para flexibilizar um pouco, uma vez que é **aproximadamente 10%**

$$x_{m+1} >= 0.89 * x_{m} + y_{m-1}$$

$$x_{m+1} <= 0.91 * x_{m} + y_{m-1}$$

3. Mês Jan temos 60 aeromoças experientes

$$y_{jan} = 60$$


4. Integralidade

$$x_{m},y_m \in Z^+ $$



In [65]:
import pyomo.environ as pyo

In [66]:
# Criação do modelo
model = pyo.ConcreteModel()

In [67]:

# Conjuntos
model.M = pyo.Set(initialize=[0,1,2,3,4,5])



In [68]:
H_m = {0:8000, 1:9000, 2:7000, 3:10000, 4:9000, 5:11000}  # Demanda de horas de voo por mês
model.H_m = pyo.Param(model.M, initialize=H_m)
Cx = 1500  # Salário de uma aeromoça regular por mês
model.Cx = pyo.Param(initialize=Cx)
Cy = 900  # Salário de uma aeromoça em treinamento por mês
model.Cy = pyo.Param(initialize=Cy)
HM = 150  # Horas de trabalho máximas por aeromoça regular por mês
model.HM = pyo.Param(initialize=HM)
HT = 100  # Horas de supervisão de uma aeromoça em treinamento
model.HT = pyo.Param(initialize=HT)
n_meses = len(H_m)  # Número de meses
model.n_meses = pyo.Param(initialize=n_meses)
x_ini = 60  # Aeromoças regulares no mês de janeiro
model.x_ini = pyo.Param(initialize=x_ini)

In [69]:

# ------------------------------
# Variáveis de Decisão
# ------------------------------

# Número de aeromoças experientes em cada mês
model.x = pyo.Var(model.M, domain=pyo.NonNegativeIntegers)

# Número de aeromoças em treinamento em cada mês
model.y = pyo.Var(model.M, domain=pyo.NonNegativeIntegers)

In [70]:

# ------------------------------
# Função Objetivo
# ------------------------------
def fo(model):
    return sum(model.Cx * model.x[m] + model.Cy * model.y[m] for m in model.M)

model.objetivo = pyo.Objective(rule=fo, sense=pyo.minimize)


In [71]:
# ------------------------------
# Restrições
# ------------------------------
# 1. Horas de Voo
def restricao_horas_voo(model, m):
    return model.HM * model.x[m] - model.HT * model.y[m] >= model.H_m[m]  

model.restricao_horas_voo = pyo.Constraint(model.M, rule=restricao_horas_voo)

# 2. Redução de aeromoças experientes ao longo dos meses
def reducao_inf(model, m):
    if m < n_meses-1:  
        return model.x[m + 1] >= 0.895 * model.x[m] + model.y[m]
    return pyo.Constraint.Skip

model.reducao_inf = pyo.Constraint(model.M, rule=reducao_inf)

def reducao_sup(model, m):
    if m < model.n_meses-1:  
        return model.x[m + 1] <= 0.91 * model.x[m] + model.y[m]
    return pyo.Constraint.Skip

model.reducao_sup = pyo.Constraint(model.M, rule=reducao_sup)

# 3. Número de aeromoças experientes no mês de janeiro
model.restricao_inicio = pyo.Constraint(expr=model.x[0] == model.x_ini)




In [72]:
# ------------------------------
# Escrita do Modelo em Arquivo
# ------------------------------
model.write("ex22.lp", io_options={"symbolic_solver_labels": True})

('ex22.lp', 3156355186944)

In [73]:
# ------------------------------
# Resolução
# ------------------------------
solver = pyo.SolverFactory("appsi_highs")
results = solver.solve(model, tee=True)

# Exibindo resultados
print("\nStatus do solver:", results.solver.status)
print("Condição de terminação:", results.solver.termination_condition)
valor_obj = pyo.value(model.objetivo)
formatted = f"{valor_obj:,.2f}"
formatted = formatted.replace(",", "X").replace(".", ",").replace("X", ".")
print("Valor da Função Objetivo: R$", formatted)

MIP  has 17 rows; 12 cols; 43 nonzeros; 12 integer variables (0 binary)
Coefficient ranges:
  Matrix [9e-01, 2e+02]
  Cost   [9e+02, 2e+03]
  Bound  [0e+00, 0e+00]
  RHS    [6e+01, 1e+04]
Presolving model
16 rows, 10 cols, 38 nonzeros  0s
12 rows, 9 cols, 32 nonzeros  0s
12 rows, 9 cols, 32 nonzeros  0s
Objective function is integral with scale 0.00333333

Solving MIP model with:
   12 rows
   9 cols (0 binary, 9 integer, 0 implied int., 0 continuous)
   32 nonzeros

Src: B => Branching; C => Central rounding; F => Feasibility pump; H => Heuristic; L => Sub-MIP;
     P => Empty MIP; R => Randomized rounding; S => Solve LP; T => Evaluate node; U => Unbounded;
     z => Trivial zero; l => Trivial lower; u => Trivial upper; p => Trivial point; X => User solution

        Nodes      |    B&B Tree     |            Objective Bounds              |  Dynamic Constraints |       Work      
Src  Proc. InQueue |  Leaves   Expl. | BestBound       BestSol              Gap |   Cuts   InLp Confl. | Lp

In [74]:

print("\nNúmero de aeromoças experientes em cada mês:")
for m in model.M:
    print(f"Mês {m + 1}: {model.x[m].value:.0f}")

print("\nNúmero de aeromoças em treinamento em cada mês:")
for m in model.M:
    print(f"Mês {m + 1}: {model.y[m].value:.0f}")


Número de aeromoças experientes em cada mês:
Mês 1: 60
Mês 2: 62
Mês 3: 59
Mês 4: 71
Mês 5: 69
Mês 6: 74

Número de aeromoças em treinamento em cada mês:
Mês 1: 8
Mês 2: 3
Mês 3: 18
Mês 4: 5
Mês 5: 12
Mês 6: 0
